First of all we will import the required libraries.

In [67]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image, display 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


To get Foursquare data, we use CLIENT_ID and CLIENT_SECRET, which we get from Foursquare. 

In [2]:
CLIENT_ID = '115N3XYYUZGF2TIJCZBVDRYHKA2IZV34GZJ4EFZKZH1GR3MG' # your Foursquare ID
CLIENT_SECRET = '2VA10XHJF4M1MB2SXVZ4U3Z540CZLD52KXY0FA1JFCUSRO3T' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 115N3XYYUZGF2TIJCZBVDRYHKA2IZV34GZJ4EFZKZH1GR3MG
CLIENT_SECRET:2VA10XHJF4M1MB2SXVZ4U3Z540CZLD52KXY0FA1JFCUSRO3T


Then we get the latitude and longitude for Aleppo and Damascus using geolocator

In [5]:
Al_address = 'Aleppo'
Dms_address = 'Damascus'
geolocator = Nominatim()
Al_location = geolocator.geocode(Al_address)
Al_latitude = Al_location.latitude
Al_longitude = Al_location.longitude
print("Aleppo latitude is {} , Aleppo longitude is {}".format(Al_latitude,Al_longitude))
Dms_location = geolocator.geocode(Dms_address)
Dms_latitude = Dms_location.latitude
Dms_longitude = Dms_location.longitude
print("Damascus latitude is {} , Damscuslongitude is {}".format(Dms_latitude,Dms_longitude))

C:\ProgramData\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Aleppo latitude is 36.19924 , Aleppo longitude is 37.1637253
Damascus latitude is 33.5130695 , Damscuslongitude is 36.3095814


Then we create the required URLs to get data from  foursquare

In [6]:
radius = 20000
LIMIT = 100
Al_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Al_latitude, Al_longitude, VERSION, radius, LIMIT)
Dms_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Dms_latitude, Dms_longitude, VERSION, radius, LIMIT)


Getting the json files of the two cities:

In [81]:
Al_results = requests.get(Al_url).json()
Dms_results = requests.get(Dms_url).json()

Then we assign relevant part of JSON to venues and tranform venues into a dataframes

In [82]:
# assign relevant part of JSON to venues
Al_venues = Al_results['response']['venues']

# tranform venues into a dataframe
Al_df = json_normalize(Al_venues)
display(Al_df.head())
Dms_venues = Dms_results['response']['venues']

# tranform venues into a dataframe
Dms_df = json_normalize(Dms_venues)
display(Dms_df.head())

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '50aaa49e4b90af0d42d5de11', 'name': 'C...",False,5859fb5b2cba225e93f9117d,NaN,SY,حلب,سوريا,NaN,90,"[حلب, حلب, سوريا]","[{'label': 'display', 'lat': 36.19918, 'lng': ...",36.199180,37.162724,NaN,حلب,Citadel Of Aleppo,v-1541593841
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4ff20855e4b0f0083af56fc3,citadel courniche,SY,حلب,سوريا,NaN,66,"[citadel courniche, حلب, حلب, سوريا]","[{'label': 'display', 'lat': 36.1994237930459,...",36.199424,37.163017,NaN,حلب,Arica Cafe,v-1541593841
2,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",False,4f574a90e4b01888b31713a2,القلعه,SY,حلب,سوريا,NaN,71,"[القلعه, حلب, حلب, سوريا]","[{'label': 'display', 'lat': 36.19867749916846...",36.198677,37.164106,NaN,حلب,القصر العدلي,v-1541593841
3,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,53b1446c498e3bbe35050ea6,ساحة سعد الله الجابري,SY,حلب,سوريا,NaN,1738,"[ساحة سعد الله الجابري, حلب, حلب, سوريا]","[{'label': 'display', 'lat': 36.206899, 'lng':...",36.206899,37.146859,NaN,حلب,مديرية الاتصالات في حلب,v-1541593841
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4cbc54d17a5d9eb0d05b31e9,Al Khandaq Street P.O. Box 13964,SY,حلب,سوريا,NaN,1199,"[Al Khandaq Street P.O. Box 13964, حلب, حلب, س...","[{'label': 'display', 'lat': 36.20503614021627...",36.205036,37.152468,NaN,حلب,Sheraton Aleppo Hotel,v-1541593841


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",False,4d090328143cb60c636c7ea6,Al Moustaqim St,SY,دمشق,سوريا,NaN,148,"[Al Moustaqim St, دمشق, دمشق, سوريا]","[{'label': 'display', 'lat': 33.51175546646118...",33.511755,36.309852,NaN,NaN,دمشق,Bab Al Hara,v-1541593841
1,"[{'id': '4bf58dd8d48988d138941735', 'name': 'M...",False,4c82011fe63376b0d0cab03d,Damascus,SY,دمشق,سوريا,NaN,325,"[Damascus, دمشق, دمشق, سوريا]","[{'label': 'display', 'lat': 33.51154891734229...",33.511549,36.306585,NaN,NaN,دمشق,Omayyad Mosque,v-1541593841
2,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",False,4d4db4869ee1a35d77eb28df,NaN,SY,دمشق,سوريا,NaN,185,"[دمشق, دمشق, سوريا]","[{'label': 'display', 'lat': 33.51331419885032...",33.513314,36.307600,NaN,NaN,دمشق,Al Oliya,v-1541593841
3,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",False,4ff9bfd7e4b02b9716414d15,NaN,SY,دمشق,سوريا,NaN,153,"[دمشق, دمشق 11211, سوريا]","[{'label': 'display', 'lat': 33.51170643105869...",33.511706,36.309797,NaN,11211,دمشق,Al-Qemarriyeh,v-1541593841
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,51c75db7498ef32120d16b85,القيمرية,SY,دمشق القديمة,سوريا,القباقبية,239,"[القيمرية (القباقبية), دمشق القديمة, دمشق, سوريا]","[{'label': 'display', 'lat': 33.51158305073101...",33.511583,36.307717,NaN,NaN,دمشق,قهوة النوفرة,v-1541593841


We need some columns from theses dataframes and not all columns, so we make new dataframes including these columns.

In [83]:
filtered_columns = ['name', 'categories'] + [col for col in Al_df.columns if col.startswith('location.')] + ['id']
Al_df_filtered = Al_df.loc[:, filtered_columns]
Dms_df_filtered = Dms_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Al_df_filtered['categories'] = Al_df_filtered.apply(get_category_type, axis=1)
Dms_df_filtered['categories'] = Dms_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Al_df_filtered.columns = [column.split('.')[-1] for column in Al_df_filtered.columns]
Dms_df_filtered.columns = [column.split('.')[-1] for column in Dms_df_filtered.columns]


Al_df_filtered = Al_df_filtered[pd.notnull(Al_df_filtered['categories'])]
Dms_df_filtered = Dms_df_filtered[pd.notnull(Dms_df_filtered['categories'])]
filtered_columns = ['name', 'categories','lat', 'lng'] 
Al_df_filtered = Al_df_filtered.loc[:, filtered_columns]
Dms_df_filtered = Dms_df_filtered.loc[:, filtered_columns]
display(Al_df_filtered.head(3))
display(Dms_df_filtered.head(3))

,name,categories,lat,lng
0,Citadel Of Aleppo,Castle,36.199180,37.162724
1,Arica Cafe,Café,36.199424,37.163017
2,القصر العدلي,Courthouse,36.198677,37.164106


,name,categories,lat,lng
0,Bab Al Hara,Historic Site,33.511755,36.309852
1,Omayyad Mosque,Mosque,33.511549,36.306585
2,Al Oliya,Middle Eastern Restaurant,33.513314,36.307600


We are interested just in hotles or in places that have Hotel in categories feild.

In [84]:
Al_df_filtered = Al_df_filtered.loc[Al_df_filtered['categories']== "Hotel", :].reset_index(drop=True)
Dms_df_filtered = Dms_df_filtered.loc[Dms_df_filtered['categories']== "Hotel", :].reset_index(drop=True)

We have 9 hotels in Aleppo

In [85]:
Al_df_filtered

,name,categories,lat,lng
0,Sheraton Aleppo Hotel,Hotel,36.205036,37.152468
1,Chahba Cham Palace Hotel Aleppo,Hotel,36.215680,37.160527
2,Coral Julia Dumna,Hotel,36.200486,37.159862
3,Monsouriya Palace Hotel,Hotel,36.195461,37.155810
4,Dedeman Hotel Aleppo,Hotel,36.215671,37.160527
5,Jdayde Hotel,Hotel,36.206952,37.156693
6,Dar Halabia,Hotel,36.198287,37.152846
7,Beit Wakil Hotel,Hotel,36.207741,37.156249
8,Ramasis Hotel,Hotel,36.204574,37.149630


We have 3 hotels in Damascus

In [86]:
Dms_df_filtered

,name,categories,lat,lng
0,Talisman 2 Hotel فندق تليسمان 2,Hotel,33.511886,36.311501
1,Shahrayar Hotel Damascus,Hotel,33.510941,36.310115
2,Shahbandar Palace Hotel,Hotel,33.513000,36.312284


Then we will show the map that includes the hotels and the center of each city.
The center location of the map will be the mean of the locations of the two cities. 

In [94]:
latitude, longitude = (Al_latitude+ Dms_latitude)/2., (Al_longitude+Dms_longitude)/2.
venues_map = folium.Map(location=[latitude, longitude], zoom_start=7) 

# add a red circle marker to represent Aleppo
folium.features.CircleMarker(
    [Al_latitude, Al_longitude],
    radius=10,
    color='red',
    popup='Aleppo',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
# add a red circle marker to represent Damascus
folium.features.CircleMarker(
    [Dms_latitude, Dms_longitude],
    radius=10,
    color='red',
    popup='Damascus',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the hotels as blue circle markers
for lat, lng, label in zip(Al_df_filtered.lat, Al_df_filtered.lng, Al_df_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
for lat, lng, label in zip(Dms_df_filtered.lat, Dms_df_filtered.lng, Dms_df_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
display(venues_map)

Then we join the two dataframes in one dataframe. The city isn't shown in the dataframe.

In [95]:
cities_grouped= pd.concat([Dms_df_filtered, Al_df_filtered])
display(cities_grouped)


,name,categories,lat,lng
0,Talisman 2 Hotel فندق تليسمان 2,Hotel,33.511886,36.311501
1,Shahrayar Hotel Damascus,Hotel,33.510941,36.310115
2,Shahbandar Palace Hotel,Hotel,33.513000,36.312284
0,Sheraton Aleppo Hotel,Hotel,36.205036,37.152468
1,Chahba Cham Palace Hotel Aleppo,Hotel,36.215680,37.160527
2,Coral Julia Dumna,Hotel,36.200486,37.159862
3,Monsouriya Palace Hotel,Hotel,36.195461,37.155810
4,Dedeman Hotel Aleppo,Hotel,36.215671,37.160527
5,Jdayde Hotel,Hotel,36.206952,37.156693
6,Dar Halabia,Hotel,36.198287,37.152846


Then we will cluster the hotels into two clusters using KMeans. 
First we drop 'name' and 'categories' columns and keep just columns tgat help us to make clustering: name , lat, and lng.
Obviosly the center of each cluster will be in one city different from the other. This will be just an experiment. In real life, there will be more than two cities, and there may be some hotels out of the cities. KMeans helps to solve these problems. 

In [96]:
kclusters = 2

cities_grouped_clustering = cities_df.drop('name', 1)
cities_grouped_clustering = cities_grouped_clustering.drop('categories', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cities_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Check cluster centers

In [97]:
kmeans.cluster_centers_.tolist()

[[36.205543196128616, 37.1560678301172],
 [33.51194230889651, 36.31130016706016]]

Then we add an column including cluster label for each hotel.

In [98]:
cities_grouped['Cluster Labels'] = kmeans.labels_

In [99]:
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
print(x,ys)

[0 1] [array([0, 1]), array([1, 3])]


Then we will show a map with different colors for different clusters or hotels in different clusters.

In [122]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cities_grouped['lat'], cities_grouped['lng'], cities_grouped['name'], cities_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
for lat, lon, poi, cluster in zip(cities_grouped['lat'], cities_grouped['lng'], cities_grouped['name'], cities_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

    
from IPython.display import IFrame
mapWidth, mapHeight = (400,500) # width and height of the displayed iFrame, in pixels
e = map_clusters.save('spst.html')
#map_clusters.create_map(path='maps/dbscanclusters.html')
IFrame('spst.html',width=700,height=350)

In [119]:
map_poi = folium.Map()
vega = folium.Vega(json.loads(densities[i].to_json()), width=350,height=200)
popup = folium.Popup(max_width=350).add_child(vega)
circle = folium.CircleMarker(location=[centers[i,0], centers[i,1]], radius=100, popup=popup)
map_poi.add_children(circle)

map_poi.create_map(path='maps/dbscanclusters.html')
IFrame('maps/dbscanclusters.html',width=700,height=350)

NameError: name 'json' is not defined